# Air Freight Time Series Data

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()

In [ ]:
import pandas as pd

we use the dataset here: https://data.transportation.gov/Aviation/International_Report_Freight/u4sg-r5vg

More illustrations: https://www.kaggle.com/parulpandey/us-international-air-traffic-data

The scheduled, charter, total columns are the total weight in tons.

In [ ]:
data_uri = "https://data.transportation.gov/api/views/u4sg-r5vg/rows.csv?accessType=DOWNLOAD"

df = pd.read_csv(data_uri)

In [ ]:
df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total
0,05/01/2008,2008,5,13930,ORD,41,16217,YUL,941,19531,AC,0,Freight,3,0,3
1,06/01/2005,2005,6,13303,MIA,33,14739,SDQ,224,20400,PT,1,Freight,0,21,21
2,09/01/2006,2006,9,13204,MCO,33,12972,LHR,493,19540,BA,0,Freight,15,0,15
3,08/01/2004,2004,8,13487,MSP,63,10292,AMS,461,19386,NW,1,Freight,479,0,479
4,03/01/2004,2004,3,12478,JFK,22,16321,ZRH,486,19805,AA,1,Freight,687,0,687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572134,03/01/2020,2020,3,13303,MIA,33,16858,IST,679,19821,TK,0,Freight,615,0,615
572135,03/01/2020,2020,3,13930,ORD,41,12395,ISL,679,19821,TK,0,Freight,888,0,888
572136,03/01/2020,2020,3,13930,ORD,41,12395,ISL,679,20370,KAQ,1,Freight,0,716,716
572137,03/01/2020,2020,3,13930,ORD,41,16858,IST,679,19821,TK,0,Freight,672,0,672


alt.Chart(...)

In [ ]:
df_daily = df[["data_dte", "airlineid"]].groupby(
    "data_dte"
).agg(
    {
        "airlineid": [
            ("counts", "count"),
            ("unique", "nunique")
        ]
    }
)
df_daily.droplevel(0,axis=1)

,counts,unique
data_dte,,
01/01/1990,1165,149
01/01/1991,1152,154
01/01/1992,1255,156
01/01/1993,1246,164
01/01/1994,1300,164
...,...,...
12/01/2015,1592,141
12/01/2016,1654,145
12/01/2017,1702,144


Build the time series of daily tonnage for all.

In [ ]:
df.head()

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total
0,05/01/2008,2008,5,13930,ORD,41,16217,YUL,941,19531,AC,0,Freight,3,0,3
1,06/01/2005,2005,6,13303,MIA,33,14739,SDQ,224,20400,PT,1,Freight,0,21,21
2,09/01/2006,2006,9,13204,MCO,33,12972,LHR,493,19540,BA,0,Freight,15,0,15
3,08/01/2004,2004,8,13487,MSP,63,10292,AMS,461,19386,NW,1,Freight,479,0,479
4,03/01/2004,2004,3,12478,JFK,22,16321,ZRH,486,19805,AA,1,Freight,687,0,687


In [ ]:
df_daily_tonnage = df.groupby(
    ["data_dte", "usg_apt", "fg_apt"]
).agg({
    "Total": [("tonnage", "sum")]
})
df_daily_tonnage = df_daily_tonnage.droplevel(0, axis=1).reset_index()
df_daily_tonnage

,data_dte,usg_apt,fg_apt,tonnage
0,01/01/1990,ADK,JCK,15
1,01/01/1990,ALB,YQX,52
2,01/01/1990,ANC,AMS,555
3,01/01/1990,ANC,ARN,38
4,01/01/1990,ANC,BRU,390
...,...,...,...,...
343243,12/01/2019,WRI,MHZ,51
343244,12/01/2019,WRI,TER,28
343245,12/01/2019,YIP,BJX,27
343246,12/01/2019,YIP,YHM,5


In [ ]:
df_daily_tonnage.groupby(
    ["usg_apt", "fg_apt"]
).agg(
    {
        "data_dte": [
            ("unique_dates", "nunique"),
            ("count_dates", "count")
        ]
    }
).droplevel(0, axis=1).sort_values(by=["unique_dates"], ascending=False).head()

unique_dates  count_dates
usg_apt fg_apt                           
DFW     MEX              369          369
SFO     FRA              369          369
        HKG              369          369
JFK     MEX              369          369
MIA     LHR              369          369

We will look into ('SFO', 'FRA')

In [ ]:
df_daily_tonnage_sfo_fra = df_daily_tonnage.loc[
    (
        df_daily_tonnage.usg_apt == "SFO"
    ) & (
        df_daily_tonnage.fg_apt == "FRA"
    )
]

## Trends Cycles, and more

In [ ]:

df_daily_tonnage_sfo_fra

,data_dte,usg_apt,fg_apt,tonnage
575,01/01/1990,SFO,FRA,735
1232,01/01/1991,SFO,FRA,884
1912,01/01/1992,SFO,FRA,877
2605,01/01/1993,SFO,FRA,1018
3318,01/01/1994,SFO,FRA,1144
...,...,...,...,...
338861,12/01/2015,SFO,FRA,601
339913,12/01/2016,SFO,FRA,705
341011,12/01/2017,SFO,FRA,869
342099,12/01/2018,SFO,FRA,1527


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8a1762a-62b2-46cf-95c0-ea77984dacfd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>